In [1]:
import numpy as np
import mne
import pandas as pd
import sys

import csv
from scipy.stats import skew, kurtosis
#import pyeeg as p
from numpy import nan
import math

In [2]:
np.set_printoptions(threshold=sys.maxsize)

In [3]:
edf = mne.io.read_raw_edf('chb01_04.edf')
header = ','.join(edf.ch_names)

Extracting EDF parameters from C:\Users\upama\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-3-6b5873c81120>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  edf = mne.io.read_raw_edf('chb01_04.edf')


In [9]:
#edf_f = edf.filter(l_freq = 4 , h_freq = 14)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 14 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 14.00 Hz
- Upper transition bandwidth: 3.50 Hz (-6 dB cutoff frequency: 15.75 Hz)
- Filter length: 423 samples (1.652 sec)



In [ ]:
#np.savetxt('your_csv_file.csv', edf.get_data().T, delimiter=',', header=header)

In [35]:
#functions for calc of stat features
# def mean_variance(df):
#     variance_vals = np.var(df)
#     return np.mean(variance_vals)

In [4]:
#mean variance
def mean_variance(df):
    variance_vals = np.var(df)
    return np.mean(variance_vals)

In [5]:
#kurtosis
def mean_kurtosis(df):
    kurtosis_vals = kurtosis(df)
    return np.mean(kurtosis_vals)

In [ ]:
#hurst
def mean_hurst(df):
    hurst_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        h_val = p.hurst(col)
        hurst_vals.append(h_val)
    return np.mean(hurst_vals)

In [6]:
#mean skewness
def mean_skewness(df):
    skew_vals = skew(df)
    return np.mean(skew_vals)

In [ ]:
#mean Petrosian Fractal Dimension
def mean_pfd(df):
    pfd_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        pfd_val = p.pfd(col)
        pfd_vals.append(pfd_val)
    return np.mean(pfd_vals)

In [8]:
#hjorth 
def mean_hjorth_mob_comp(df):
    mob_vals = []
    comp_vals = []
    for col in df.columns:
        col = df[col].to_numpy()
        mob_col, comp_col = p.hjorth(col)
        mob_vals.append(mob_col)
        comp_vals.append(comp_col)
    return np.mean(mob_vals), np.mean(comp_vals)

In [ ]:
#mean spectral entropy
def mean_spectral_entropy(df):
    psd_vals = all_psd(df)
    power_ratio = []
    sum_psd_vals = sum_psd(df)
    for val in psd_vals:
        power_ratio.append(val/sum_psd_vals)
    bands = [0,4,8,12,30,45]
    Fs = 256
    spec_entropy = p.spectral_entropy(df, bands, Fs, power_ratio)
    return spec_entropy

In [51]:
seizure_start = 1467
seizure_end = 1494
#window function to work over data and calculate features
def add_feature(edf_file,start,duration):
    d = edf_file[:,256*start:256*(start+duration)]
    row_to_add = []
    mean_var = mean_variance(d[0])
    
    if start in range(seizure_start,(seizure_end,-1)):
        t = 1
    else:
        t = 0
    row_to_add.append(mean_var)
    row_to_add.append(start)
    row_to_add.append(t)
    return row_to_add

In [70]:
df = pd.DataFrame() #columns = ['mean_var','start','target']

In [71]:
#driver code
start = temp = 1397
duration = 10
end = 1574
while temp<end:
    row = add_feature(edf,temp,duration)
    df=df.append(pd.Series(row),ignore_index=True)
    temp = temp + duration

In [72]:
df

,0,1,2
0,1.045350e-09,1397.0,0.0
1,1.089297e-09,1407.0,0.0
2,1.245100e-09,1417.0,0.0
3,1.313426e-09,1427.0,0.0
4,1.010236e-09,1437.0,0.0
5,1.088562e-09,1447.0,0.0
6,3.048230e-09,1457.0,0.0
7,1.247817e-08,1467.0,1.0
8,1.114856e-08,1477.0,1.0
9,1.315812e-08,1487.0,1.0


In [10]:
data = mne.io.read_raw_edf('chb01_04.edf', preload=True)
df = data.to_data_frame()
df = df[0:20][:]
df

Extracting EDF parameters from C:\Users\upama\chb01_04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


<ipython-input-10-db914c97691a>:1: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf('chb01_04.edf', preload=True)


,time,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,...,F8-T8,T8-P8-0,P8-O2,FZ-CZ,CZ-PZ,P7-T7,T7-FT9,FT9-FT10,FT10-T8,T8-P8-1
0,0,93.186813,57.240537,-77.948718,-2.539683,78.339438,20.512821,-15.433455,-13.479853,84.200244,...,-50.598291,-2.930403,44.346764,-30.280830,58.803419,78.339438,-18.168498,-56.849817,-15.433455,-2.930403
1,4,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,...,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360
2,8,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,...,0.195360,0.195360,-0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360,0.195360
3,12,-0.195360,0.195360,0.195360,0.195360,-0.195360,0.195360,0.195360,0.586081,-0.195360,...,0.976801,0.195360,-1.367521,0.195360,0.195360,0.195360,-0.195360,0.195360,0.195360,0.195360
4,16,0.976801,-0.195360,0.195360,0.195360,0.976801,0.195360,0.976801,-0.976801,0.976801,...,-2.148962,-0.195360,3.321123,0.195360,0.976801,0.195360,2.148962,-0.586081,-0.195360,-0.195360
5,20,2.148962,-0.586081,0.195360,0.195360,2.539683,0.586081,0.195360,-2.148962,2.930403,...,-4.102564,-0.586081,6.837607,0.586081,0.586081,0.195360,2.148962,-0.976801,0.195360,-0.586081
6,23,0.195360,0.976801,0.195360,-0.976801,0.586081,0.976801,-2.539683,1.367521,0.586081,...,3.321123,1.758242,-3.711844,0.195360,-1.758242,0.195360,-4.493284,2.148962,1.758242,1.758242
7,27,-2.148962,2.539683,0.195360,-0.195360,-2.148962,0.586081,-2.148962,4.102564,-2.539683,...,9.181929,2.930403,-13.479853,0.195360,-1.367521,0.195360,-7.228327,3.711844,1.758242,2.930403
8,31,0.586081,-0.195360,0.195360,0.976801,0.195360,0.195360,3.711844,-2.539683,0.195360,...,-2.148962,-0.586081,2.930403,0.976801,3.321123,0.195360,4.884005,-1.758242,-1.367521,-0.586081
9,35,4.493284,-2.539683,0.195360,0.195360,4.102564,0.976801,4.493284,-6.837607,4.884005,...,-12.698413,-3.321123,20.122100,1.758242,4.102564,0.195360,10.354090,-4.493284,-1.758242,-3.321123


In [13]:
row_to_add = []
        
mean_var = mean_variance(df)
mean_skew = mean_skewness(df)
#mean_h_mob, mean_h_comp = mean_hjorth_mob_comp(df)
#mean_hurst_val = func_mean_hurst(df)
mean_k = mean_kurtosis(df)


row_to_add.append(mean_skew)  
row_to_add.append(mean_var)
row_to_add.append(mean_k)
#row_to_add.append(mean_h_mob)  
#row_to_add.append(mean_h_comp)
#row_to_add.append(mean_hurst_val)

row_to_add

[0.3001244458926663, 236.7253248729179, 2.3938622662388394]

In [32]:
df_f = pd.DataFrame()
start = temp = 2956
duration = 10
end = 3076
while temp<end:
    row = add_feature(edf_f,temp,duration)
    df_f=df_f.append(pd.Series(row),ignore_index=True)
    temp = temp + duration

In [33]:
df_f

,0,1
0,2.861505e-10,2956.0
1,3.494730e-10,2966.0
2,4.145274e-10,2976.0
3,3.188420e-10,2986.0
4,1.965291e-09,2996.0
5,2.459066e-09,3006.0
6,2.538410e-09,3016.0
7,1.361507e-09,3026.0
8,9.248479e-10,3036.0
9,9.684857e-10,3046.0


In [6]:
f = 2095
print(round(f,-1))

2100
